In [7]:
pip install pprint

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [8]:
import ijson
import random
import re
import pandas as pd
import json
from tqdm import tqdm
import pprint

filename = "pyedur_full.jsonl"

In [9]:
def print_line(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for i in range(random.randint(0,1630607)):
            f.readline()
        pprint.pp(f.readline())


def num_rows(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for _ in f:
            count += 1

    print(f'Total number of rows: {count}')

In [5]:
num_rows(filename)

Total number of rows: 1630607


In [6]:
print_line(filename)

{"prompt": "You are given a question that requires some input and output variables as follows:\n\nGiven a list of integers, how can we determine the smallest number using different divide-and-conquer strategies, or calculate the sum of elements at positions that are multiples of 3, or find the r-th root of a number with a specified precision?\n\nThe input and output requirements are as follows:\n\nInput:\n  `data` (list): A list of integers. For the 'root' operation, the list should contain exactly three elements: the number `n`, the root `r`, and the precision `p`.\n  `operation` (str): A string indicating the operation to perform. Valid values are 'min', 'minChip', 'sumMultiples', and 'root'.\n\nOutput:\n  `return` (int or float): The result of the specified operation. For 'min', 'minChip', and 'sumMultiples', it returns an integer. For 'root', it returns a float.\n\nGiven the following input:\n\n{'data': [-52, 83, 78, -31, -100, -56, 33, -32, -93, 38], 'operation': 'sumMultiples'}\n

In [5]:
import json
import tiktoken

def average_prompt_tokens(file_path, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    total_tokens = 0
    total_prompts = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                prompt = obj.get("prompt", "")
                token_count = len(enc.encode(prompt))
                total_tokens += token_count
                total_prompts += 1
            except json.JSONDecodeError:
                continue  # Skip malformed lines

    if total_prompts == 0:
        print("No valid prompts found.")
        return

    average = total_tokens / total_prompts
    print(f"Average number of tokens in 'prompt': {average:.2f}")


In [6]:
average_prompt_tokens(filename)


Average number of tokens in 'prompt': 809.38


In [8]:
import json
import ast

# Extract code after the marker
def extract_code(prompt):
    marker = "You can refer to this code to guide your reasoning but not copy spans of code directly."
    if marker in prompt:
        return prompt.split(marker, 1)[-1].strip()
    return ""

# Convert code to AST
def code_to_ast(code):
    try:
        tree = ast.parse(code)
        return ast.dump(tree)
    except SyntaxError as e:
        print(f"SyntaxError in parsing code: {e}")
        return None

def process_jsonl(input_path, output_path, buffer_size=5000):
    buffer = []

    # First, count total lines for tqdm
    with open(input_path, 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)

    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        for idx, line in enumerate(tqdm(infile, total=total_lines, desc="Processing JSONL")):
            try:
                data = json.loads(line)

                # Phase 1: Extract code
                prompt = data.get("prompt", "")
                code = extract_code(prompt)
                data["code"] = code

                # Phase 2: Convert to AST
                ast_representation = code_to_ast(code)
                if ast_representation is not None:
                    data["ast"] = ast_representation

                buffer.append(json.dumps(data))

                # Write in chunks
                if len(buffer) >= buffer_size:
                    outfile.write("\n".join(buffer) + "\n")
                    buffer.clear()

            except Exception as e:
                print(f"[Line {idx}] Error: {e}")
                continue

        # Write any remaining lines
        if buffer:
            outfile.write("\n".join(buffer) + "\n")


In [11]:
ast_dataset = "ast-pyedur_full.jsonl"
# process_jsonl(filename,ast_dataset)

In [12]:
print_line(ast_dataset)
num_rows(ast_dataset)

('{"prompt": "You are given a question that requires some input and output '
 'variables as follows:\\n\\nGiven the dimensions of a 3D volume in pixels, '
 'the A/pixel for the generated map, and the type of model to create (random '
 'or specified), how many Gaussian blobs are used to construct the model, and '
 'what are the dimensions of the resulting model?\\n\\nThe input and output '
 'requirements are as follows:\\n\\nInput:\\n- `dimensions` (list of int): The '
 'dimensions of the volume in pixels [x, y, z].\\n- `apix` (float): The '
 'A/pixel for the generated map.\\n- `model_type` (str): The type of model to '
 "create, either 'r' for random or 's' for specified.\\n- `blobs` (list of "
 'list of float): The parameters for each blob. For random models, this is '
 'generated internally. For specified models, this is provided by the user. '
 'Each blob is a list of 6 floats representing [r, theta, z, dx, dy, '
 'dz].\\n\\nOutput:\\n- `return` (dict): A dictionary containing the '

In [11]:
import ast
import json

def extract_code_to_ast(jsonl_line):
    """
    Extracts code from a JSONL line (from CodeI/O dataset) and returns its AST as a string.
    """
    # Parse the JSON line
    data = json.loads(jsonl_line)
    code_text = data.get("code", "")
    
    # Extract code (using your marker logic)
    marker = "You can refer to this code to guide your reasoning but not copy spans of code directly."
    if marker in code_text:
        code = code_text.split(marker, 1)[-1].strip()
    else:
        code = code_text.strip()
    
    # Parse the code into an AST
    try:
        tree = ast.parse(code)
        return ast.dump(tree)  # Convert AST to a string representation
    except SyntaxError as e:
        print(f"SyntaxError in parsing code: {e}")
        return None

def process_jsonl_2(input_path, output_path, buffer_size=5000):
    buffer = []

    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        for idx, line in enumerate(infile):
            try:
                data = json.loads(line)
                ast_representation = extract_code_to_ast(line)
                if ast_representation is not None:
                    data["ast"] = ast_representation
                buffer.append(json.dumps(data))

                # Write in chunks
                if len(buffer) >= buffer_size:
                    outfile.write("\n".join(buffer) + "\n")
                    buffer.clear()

            except Exception as e:
                print(f"[Line {idx}] Error: {e}")
                continue

        # Write any remaining lines
        if buffer:
            outfile.write("\n".join(buffer) + "\n")

In [12]:
ast_name = "ast-pyedur_full.jsonl"
process_jsonl_2(sep_code_name, ast_name)

[Line 1114999] Error: [Errno 28] No space left on device
[Line 1115000] Error: [Errno 28] No space left on device
[Line 1115001] Error: [Errno 28] No space left on device
[Line 1115002] Error: [Errno 28] No space left on device
[Line 1115003] Error: [Errno 28] No space left on device
[Line 1115004] Error: [Errno 28] No space left on device
[Line 1115005] Error: [Errno 28] No space left on device
[Line 1115006] Error: [Errno 28] No space left on device
[Line 1115007] Error: [Errno 28] No space left on device
[Line 1115008] Error: [Errno 28] No space left on device
[Line 1115009] Error: [Errno 28] No space left on device
[Line 1115010] Error: [Errno 28] No space left on device
[Line 1115011] Error: [Errno 28] No space left on device
[Line 1115012] Error: [Errno 28] No space left on device
[Line 1115013] Error: [Errno 28] No space left on device
[Line 1115014] Error: [Errno 28] No space left on device
[Line 1115015] Error: [Errno 28] No space left on device
[Line 1115016] Error: [Errno 28

KeyboardInterrupt: 

In [10]:
# print_line(ast_name)
num_rows(ast_name)

Total number of rows: 800000


In [13]:
with open("example.jsonl", "r") as f:
    line = f.readline()
    ast_tree = extract_code_to_ast(line)

# Now you can inspect the AST
if ast_tree:
    print(ast.dump(ast_tree, indent=2))  # Pretty-print the AST

TypeError: expected AST, got 'str'